In [ ]:
from dolfin import *
from mshr import *
from rbnics.backends.dolfin.wrapping import counterclockwise
from rbnics.shape_parametrization.utils.symbolic import VerticesMappingIO

In [ ]:
# reference value
m0 = 0.05

In [ ]:
# Define domain
beam = Rectangle(Point(0., 0.2),Point(3., 0.5))
left_support = Polygon([Point(0.05, 0.), Point(0.25, 0.), Point(0.15, 0.2)])
right_support = Polygon([Point(2.75, 0.), Point(2.95, 0.), Point(2.85, 0.2)])
domain = beam + left_support + right_support

In [ ]:
# Define vertices mappings of affine shape parametrization. These will be used
# to partition the mesh in subdomains.
vertices_mappings = [
    {
        ("0.05","0.0"): ("mu[0]", "0.0"),
        ("0.25","0.0"): ("mu[0]+0.2", "0.0"),
        ("0.15","0.2"): ("mu[0]+0.1", "0.2")
    }, #subdomain 1
    {
        ("2.75","0.0"): ("2.8-mu[0]", "0.0"),
        ("2.95","0.0"): ("3.0-mu[0]", "0.0"),
        ("2.85","0.2"): ("2.9-mu[0]", "0.2")
    }, #subdomain 2
    {
        ("0.0", "0.2"): ("0.0", "0.2"),
        ("0.15", "0.2"): ("mu[0]+0.1", "0.2"),
        ("0.0","0.5"): ("0.0", "0.5")
    }, #subdomain 3
    {
        ("0.0","0.5"): ("0.0","0.5"),
        ("0.15", "0.2"): ("mu[0]+0.1","0.2"),
        ("1.5","0.2"): ("1.5","0.2")
    }, #subdomain 4
    {
        ("0.0", "0.5"): ("0.0","0.5"),
        ("1.5","0.2"): ("1.5","0.2"),
        ("3.0","0.5"): ("3.0","0.5")
    }, #subdomain 5
    {
        ("1.5","0.2"): ("1.5","0.2"),
        ("2.85", "0.2"): ("2.9-mu[0]","0.2"),
        ("3.0","0.5"): ("3.0","0.5")
    }, #subdomain 6
    {
        ("2.85", "0.2"): ("2.9-mu[0]","0.2"),
        ("3.0","0.2"): ("3.0","0.2"),
        ("3.0","0.5"): ("3.0","0.5")
    }, #subdomain 7
]

In [ ]:
# Create mesh
for i, vertices_mapping in enumerate(vertices_mappings):
    subdomain_i = Polygon([Point(*[float(coord) for coord in vertex]) for vertex in counterclockwise(vertices_mapping.keys())])
    domain.set_subdomain(i + 1, subdomain_i)
mesh = generate_mesh(domain, 50)
plot(mesh)

In [ ]:
# Create subdomains
subdomains = MeshFunction("size_t", mesh, 2, mesh.domains())
plot(subdomains)

In [ ]:
# Create boundaries

#beam
class BottomLeftOut(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[1]-0.2) < DOLFIN_EPS and x[0]<= m0+0.1
class BottomLeftIn(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and ((abs(x[1]-0.2) < DOLFIN_EPS and x[0]>= m0+0.1 and x[0]<=1.5) or ((x[0]-(m0+0.1))**2+(x[1]-0.2)**2<DOLFIN_EPS))
class BottomRightIn(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[1]-0.2) < DOLFIN_EPS and x[0]>=1.5 and x[0]<=2.9-m0
class BottomRightOut(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[1]-0.2) < DOLFIN_EPS and x[0]>=2.9-m0   and x[0]<=3
class Top(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[1]-0.5) < DOLFIN_EPS
class Left(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[0]) < DOLFIN_EPS
class Right(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[0]-3) < DOLFIN_EPS

#left support
class LSBottom(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[1]) < DOLFIN_EPS and x[0]<1.5
class LSLeft(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and x[1]<=0.2 and abs(x[1]-2*x[0]+0.1) < DOLFIN_EPS
class LSRight(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and x[1]<=0.2 and abs(x[1]+2*x[0]-0.5) < DOLFIN_EPS
    
#right support
class RSBottom(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and abs(x[1]) < DOLFIN_EPS and x[0]>1.5
class RSLeft(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and ((x[1]<0.2 and x[1] > 0 and x[0] >= 2.8-m0 and x[0]<= 2.9-m0) or ((x[0]-2.85)**2+(x[1]-0.2)**2)<DOLFIN_EPS or ((x[0]-2.75)**2+(x[1]-0)**2)<DOLFIN_EPS)
class RSRight(SubDomain):
    def inside(self,x,on_boundary):
        return on_boundary and ((x[1]<0.2 and x[1] > 0 and x[0] >= 2.9-m0 and x[0]<= 3-m0) or ((x[0]-2.85)**2+(x[1]-0.2)**2)<DOLFIN_EPS or ((x[0]-2.95)**2+(x[1]-0)**2)<DOLFIN_EPS)

boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundaries.set_all(0)

lsBottom=LSBottom()
lsBottom.mark(boundaries,1)
rsBottom=RSBottom()
rsBottom.mark(boundaries,2)
bottomLeftOut=BottomLeftOut()
bottomLeftOut.mark(boundaries, 3)
bottomLeftIn=BottomLeftIn()
bottomLeftIn.mark(boundaries, 4)
bottomRightIn=BottomRightIn()
bottomRightIn.mark(boundaries,5)
bottomRightOut=BottomRightOut()
bottomRightOut.mark(boundaries,6)

top=Top()
top.mark(boundaries,7)
left=Left()
left.mark(boundaries,8)
right=Right()
right.mark(boundaries,9)
lsLeft=LSLeft()
lsLeft.mark(boundaries,10)
lsRight=LSRight()
lsRight.mark(boundaries,11)
rsRight=RSRight()
rsRight.mark(boundaries,12)
rsLeft=RSLeft()
rsLeft.mark(boundaries,13)

In [ ]:
# Save to xml file
VerticesMappingIO.save_file(vertices_mappings, ".", "bridge_vertices_mapping.vmp")
File("bridge.xml") << mesh
File("bridge_physical_region.xml") << subdomains
File("bridge_facet_region.xml") << boundaries
XDMFFile("bridge.xdmf").write(mesh)
XDMFFile("bridge_physical_region.xdmf").write(subdomains)
XDMFFile("bridge_facet_region.xdmf").write(boundaries)